In [2]:
import os.path as osp
import os

#torch 전에 할댕해줘야함
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [3]:


import torch
from tqdm import tqdm

from torch_geometric.datasets import AmazonBook, MovieLens100K
from torch_geometric.nn import LightGCN
from torch_geometric.utils import degree
from torch_geometric.data import InMemoryDataset

In [4]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [29]:
# ml 1m 
path = osp.join('./', 'data', 'ML1M')
dataset = MovieLens100K(path)
dataset.process()
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['movie'].num_nodes
data = data.to_homogeneous().to(device)
# 이새끼 호모지니어스로 바꾸면 엣지 라벨 순서가 바뀌어 버림
data.edge_label_index = torch.stack([data.edge_label_index[1],data.edge_label_index[0]], dim = 0)
data.edge_label_index

tensor([[   5,    9,   11,  ...,  933,    9,  681],
        [1682, 1682, 1682,  ..., 2140, 2141, 2143]], device='cuda:0')

In [30]:
# 아마존
path = osp.join('./', 'data', 'Amazon')
dataset = AmazonBook(path)
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['book'].num_nodes
data = data.to_homogeneous().to(device)


In [6]:
# Gowalla
from .datasets import Gowalla
path = osp.join('./', 'data', 'Gowalla')
dataset = Gowalla(path)
dataset.process()
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['item'].num_nodes
data = data.to_homogeneous().to(device)

/tmp/ipykernel_3560841/1471277341.py:127: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327249/work/torch/csrc/utils/tensor_new.cpp:278.)
  index = torch.tensor([rows, cols])


In [7]:
dataset[0]

HeteroData(
  user={ num_nodes=29858 },
  item={ num_nodes=40988 },
  (user, rates, item)={
    edge_index=[2, 821971],
    edge_label_index=[2, 205493],
  },
  (item, rated_by, user)={ edge_index=[2, 821971] }
)

In [12]:
# Use all message passing edges as training labels:
batch_size = 1024
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=3
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
def train():
    total_loss = total_examples = 0

    for index in train_loader:
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
            lambda_reg = 0.0001
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

In [14]:
@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = min(start + batch_size, num_users)
        logits = user_emb[start:end] @ book_emb.t()
        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
                train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                        data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                                num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

In [11]:
best = 0
best_model = model.get_embedding(data.edge_index).detach()
best_init = model.embedding.weight.detach()
p = 0
for epoch in tqdm(range(0, 1000)):
    loss = train()
    precision, recall = test(k=20)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Precision@20: '
            f'{precision:.4f}, Recall@20: {recall:.4f}')
        if recall > best:
            p = 0
            best_model = model.get_embedding(data.edge_index).detach()
            best_init = model.embedding.weight.detach()
            best = recall
        else:
            p += 1
            if p > 10:
                break

  0%|          | 1/1000 [00:18<5:05:58, 18.38s/it]

Epoch: 000, Loss: 0.3528, Precision@20: 0.0209, Recall@20: 0.0702


  1%|          | 11/1000 [03:23<5:06:18, 18.58s/it]

Epoch: 010, Loss: 0.0931, Precision@20: 0.0321, Recall@20: 0.1109


  2%|▏         | 21/1000 [06:31<5:06:26, 18.78s/it]

...

 31%|███       | 311/1000 [1:36:38<3:33:34, 18.60s/it]

Epoch: 310, Loss: 0.0228, Precision@20: 0.0553, Recall@20: 0.1893


 32%|███▏      | 321/1000 [1:39:44<3:30:36, 18.61s/it]

Epoch: 320, Loss: 0.0226, Precision@20: 0.0555, Recall@20: 0.1903


 33%|███▎      | 328/1000 [1:41:54<3:28:20, 18.60s/it]